In [72]:
import pandas as pd
import datadotworld as dw
import seaborn as sns
from datetime import date

%matplotlib inline

In [3]:
# weather_ds = dw.load_dataset('ride-austin/austin-weather-june-2-2016-april-13-2017')
# ride_first_ds = dw.load_dataset('ride-austin/ride-austin-june-6-april-13')
# ride_second_ds = dw.load_dataset('ride-austin-june-6-april-13-part-2')

In [10]:
weather_df = pd.read_csv('Weather_Data.csv', index_col=0)
ride_data_first_df = pd.read_csv('Rides_DataA.csv', index_col=0)
ride_data_second_df = pd.read_csv('Rides_DataB.csv', index_col=0)
joined_df = weather_df.join(ride_data_first_df).join(ride_data_second_df)

In [38]:
joined_df[:len(joined_df) / 2].to_csv('merged_ride_weather_data_part_1.csv', index=False)
joined_df[len(joined_df) / 2:].to_csv('merged_ride_weather_data_part_2.csv', index=False)

In [101]:
joined_df['created_date'][0]

'2016-06-04 00:10:19-05:00'

In [44]:
joined_df.columns.values

array(['HOURLYVISIBILITY', 'HOURLYDRYBULBTEMPC', 'HOURLYRelativeHumidity',
       'HOURLYWindSpeed', 'HOURLYWindDirection', 'HOURLYPrecip',
       'DAILYMaximumDryBulbTemp', 'DAILYMinimumDryBulbTemp',
       'DAILYDeptFromNormalAverageTemp', 'DAILYAverageRelativeHumidity',
       'DAILYSunrise', 'DAILYSunset', 'DAILYPrecip',
       'DAILYAverageWindSpeed', 'DAILYPeakWindSpeed', 'started_on',
       'created_date', 'updated_date', 'completed_on',
       'distance_travelled', 'end_location_lat', 'end_location_long',
       'driver_rating', 'rider_rating', 'active_driver_id', 'charity_id',
       'requested_car_category', 'free_credit_used', 'surge_factor',
       'start_location_long', 'start_location_lat', 'rider_id',
       'round_up_amount', 'promocode_redemption_id', 'driver_reached_on',
       'base_fare', 'total_fare', 'rate_per_mile', 'rate_per_minute',
       'time_fare', 'driver_accepted_on', 'esimtated_time_arrive',
       'tipped_on', 'tip', 'driving_time_to_rider',
       'di

In [95]:
def get_time_format(time):
    info = time.split(' ')
    year, month, day = map(int, info[0].split('-'))
    return date(year, month, day)

def get_first_time(row):
    register_date = row['created_date']
    first_date = row['completed_on']
    r_date = get_time_format(register_date)
    f_date = get_time_format(first_date)
    return 1

In [107]:
joined_df['get_first_time'] = joined_df.apply(lambda row: get_first_time(row), axis=1)

In [111]:
joined_df['get_first_time']

1494125